In [ ]:
# Create file directories
!mkdir data_buffers
!mkdir models

In [ ]:
# %%writefile featurise_phc_dataset.py

import pandas as pd
import numpy as np
import phclib as pp
import os
import warnings
warnings.filterwarnings("ignore")

print('Generating features....')
start = pd.Timestamp.now()

# Load dataset

df = pd.read_parquet('phc_data.pqt')
weather_internal = pd.read_parquet('weather_data.pqt')

# Convert to daily format to append daily data such as fast-logging, weather or other (here we just use weather)

meters_list = pp.chunks(df.meterref.unique(), 35) # Break the data into chunks for processing to fit into memory

df_list = []

for i, meters in enumerate(meters_list):
    print('{}/{}   '.format(i+1,len(meters_list)), end='\r')
    df_subset = df[df.meterref.isin(meters)].copy()

    # Create a multiplier dataframe in order to set up to perform approximate merge on it

    multiplied_weather_df_list = []
    for meter in df_subset['meterref'].unique():
        df_part = weather_internal[['dateofread_dt']].copy()
        df_part['meterref'] = meter
        multiplied_weather_df_list.append(df_part)
    multiplied_weather_df = pd.concat(multiplied_weather_df_list)
    del multiplied_weather_df_list

    # Do a backwards approximate merge to associate any days of weather data to appropriate meter window
    combined_df = pd.merge_asof(multiplied_weather_df.sort_values('dateofread_dt'),
                                df_subset[['prevreaddate', 'meterref','currreaddate','daily_consumption', 'prev_daily_consumption']].sort_values('prevreaddate'),
                                by='meterref',
                                left_on='dateofread_dt',
                                right_on='prevreaddate',
                                direction='backward')

    # Remove any data outside the meter read window
    combined_df = combined_df[(combined_df.prevreaddate >= combined_df.dateofread_dt.min()) & (combined_df.currreaddate > combined_df.dateofread_dt)]
    combined_df = combined_df.drop_duplicates(['meterref','prevreaddate','dateofread_dt'])
    
    combined_df = combined_df.sort_values(['meterref','dateofread_dt']).reset_index(drop=True)

    combined_df['cum_prev_daily_consumption2'] = combined_df.groupby('meterref')['prev_daily_consumption'].cumsum() / combined_df.groupby('meterref')['prev_daily_consumption'].cumcount()

    # Merge in weather data
    combined_df = combined_df.merge(weather_internal, how='left',on='dateofread_dt')

    # Save daily data buffers to be used later
    combined_df[['prevreaddate', 'meterref','dateofread_dt','currreaddate','daily_consumption', 'cum_prev_daily_consumption2']].to_parquet(os.getcwd()+'/data_buffers/daily_subset_part_{}'.format(i))

    # Create a variable for each date
    for date in combined_df['dateofread_dt'].unique():
        combined_df['{}'.format(date)] =  (combined_df['dateofread_dt'] == date).astype(np.uint8)

    #Remove fields no longer needed, as they are present in the original data subset
    combined_df.drop(['daily_consumption','currreaddate', 'prev_daily_consumption'],1,inplace=True)

    # Aggregate back to the meter read window
    combined_df = pp.compress_df(combined_df)
    agg_dict = {**{feat:'mean' for feat in weather_internal.drop('dateofread_dt',1)} ,
                **{feat: 'sum' for feat in [c for c in combined_df if '20' in c]},
                **{'cum_prev_daily_consumption2': 'mean'}}

    combined_df_agg = combined_df.groupby(['meterref','prevreaddate']).agg(agg_dict).reset_index()

    # Merge in the rest of the original data
    df_subset = df_subset.merge(combined_df_agg, on=['meterref','prevreaddate'])
    df_list.append(df_subset)

final = pd.concat(df_list)
del df_list

# Save meterref breakdown for future use
pp.save_obj(meters_list, 'meterref_split_list')
    

# One-hot-encode categoricals
for c in ['b_class','age','type','storeys','classdesc','primarydesc','secondarydesc',
          'tertiarydesc','quaternarydesc','acorntype', 'acorncat', 'acorngroup']:
    final = pd.concat([final, pd.get_dummies(final[c], prefix=c)], 1)
    final.drop(c, 1, inplace=True)

final = final.sort_values(['meterref','prevreaddate'])
final.columns = pp.pythonify_cols(final.columns)
    
# Apply exclusions
final = final[(final['reading_lag_prev'] >= 0) & (final['reading_lag_prev'] < 90)] # to bridge gap of small missing reading windows
final = final[final['reading_lag_prev_start'] <= 200] # Select cases where previous reading window was not too long ago
final = final[final['reading_window_days'] < 200] # Use under 5 month intervals only in order to get variance within the year
final = final[(final['daily_consumption'] >= 0) & (final['daily_consumption'] < 200)] # Ensure no 99999 and other corruped readings
final = final[(final['prev_daily_consumption'] >= 0) & (final['prev_daily_consumption'] < 200)]# Ensure no 99999 and other corruped readings

#Save data
final.to_parquet('featurised_phc_data.pqt')

print('Finished featurisation. Data shape: {} / Time elapsed (minutes): {}'.format(final.shape, (pd.Timestamp.now() - start) / pd.Timedelta(1, 'm')))

In [ ]:
%reset -f

In [ ]:
# %%writefile build_daily_variance_models.py

from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
import pandas as pd
import numpy as np
import os
import phclib as pp
import pickle
import warnings
warnings.filterwarnings("ignore")

print('Training models...')
start = pd.Timestamp.now()

# Load relevant data
weather_internal = pd.read_parquet('weather_data.pqt', columns=['dateofread_dt'])
model_data = pd.read_parquet('featurised_phc_data.pqt')
model_data = model_data[(model_data.prevreaddate >= '2018-02-01 00:00:00')]

# Decide on modelling window
modelling_window_dates = weather_internal[(weather_internal.dateofread_dt >= '2018-03-01 00:00:00') & (weather_internal.dateofread_dt < '2020-07-29 00:00:00')].dateofread_dt.unique()
modelling_window_dates = pp.pythonify_cols(modelling_window_dates.astype(str))

# Exclude features not to model on - the idea is to only get variance based on seasonality, weather and static data
exclude = ['prevreaddate','meterref','dateofread_dt','currreaddate','daily_consumption',
          'reading_window_days', 'reading_lag_prev_start'] + pp.pythonify_cols([x.lower() for x in model_data if '000000000' in x])

# Train models in a loop
for i, date in enumerate(modelling_window_dates):
    
    model_data_subset = model_data[model_data[str(date)] >= 1].copy()
    print('Model number, date and data shape: ', i+1, date, model_data_subset.shape[0], end='\r')
    use_cols = [c for c in model_data_subset if c not in exclude]

    model_data_subset = model_data_subset.reset_index(drop=True)

    # Create train / test split
    X_train, X_test, y_train, y_test = train_test_split(model_data_subset[use_cols], model_data_subset['daily_consumption'],
                                                        test_size=0.1, random_state=1)

    # Initiate xgb model with parameters that are likely to overfit as we'll be using an automated stopping point
    XGB_model = XGBRegressor(learning_rate=0.075,
                             max_depth=5,
                             n_estimators=250,
                             n_jobs=31,
                             objective='reg:squarederror')
    
    # Fit with automated stopping based on the test set
    XGB_model.fit(X_train,
                  y_train,
                  verbose=False,
                  early_stopping_rounds=10,
                  eval_metric='rmse',
                  eval_set=[(X_test, y_test)])
    
    
    train_score = XGB_model.score(X_train, y_train)
    test_score = XGB_model.score(X_test, y_test)

    model_data_subset['predicted'] = XGB_model.predict(model_data_subset[use_cols])
    total_pred = model_data_subset['predicted'].sum()
    total_act = model_data_subset['daily_consumption'].sum()
    sample_shape = model_data_subset.shape[0]
    res = {date: {'model': XGB_model,
                           'train_score': train_score, 
                           'test_score': test_score,
                           'total_pred': total_pred,
                           'total_actual': total_act,
                           'sample_shape': sample_shape
                          }}

    pp.save_obj(res, os.getcwd()+'/models/model_{}.pkl'.format(date), full_path=True)
# pp.save_obj(res, os.getcwd()+'/fall_back_model_{}.pkl'.format(date), full_path=True)
print('Saving to disk. Successfully built {} models over {} minutes'.format(len(modelling_window_dates), (pd.Timestamp.now() - start) / pd.Timedelta(1, 'm')))
pp.save_obj(use_cols, 'all_feats')
pp.save_obj(modelling_window_dates, 'modelling_window')


stats = [list(pp.load_obj(os.getcwd()+'/models/{}'.format(file), full_path=True).values())[0]['train_score'] for file in os.listdir(os.getcwd()+'/models/')]
print('Train Score: ', np.mean(stats))

stats = [list(pp.load_obj(os.getcwd()+'/models/{}'.format(file), full_path=True).values())[0]['test_score'] for file in os.listdir(os.getcwd()+'/models/')]
print('Test Score: ', np.mean(stats))

stats = [list(pp.load_obj(os.getcwd()+'/models/{}'.format(file), full_path=True).values())[0]['total_pred'] for file in os.listdir(os.getcwd()+'/models/')]
print('Total prediction: ', np.mean(stats))

stats = [list(pp.load_obj(os.getcwd()+'/models/{}'.format(file), full_path=True).values())[0]['total_actual'] for file in os.listdir(os.getcwd()+'/models/')]
print('Total actual: ', np.mean(stats))

In [ ]:
%reset -f

In [ ]:
# %%writefile daily_redistribution_plus_featurisation.py

import pandas as pd
import numpy as np
import phclib as pp
import os
import warnings
warnings.filterwarnings("ignore")

print('Redistributing the dependent variable (daily consumption) based on the daily models...')
start = pd.Timestamp.now()

# Load files
meters_list = pp.load_obj('meterref_split_list')
df = pd.read_parquet('phc_data.pqt')
all_feats = pp.load_obj('all_feats')
modelling_window_dates = pp.load_obj('modelling_window')
modelling_window_dates_date_format = pd.to_datetime(pd.Series(modelling_window_dates).str.replace('_','-')).to_list()
weather_internal = pd.read_parquet('weather_data.pqt')

# Load the buffered daily data from before and merge in all the rest of the data
for i, meters in enumerate(meters_list):
    print('{}/{}{}'.format(i+1,len(meters_list), ' '*20), end='\r')
    df_subset = df[df.meterref.isin(meters)].copy()
    if df_subset.shape[0] == 0:
        continue
    combined_df = pd.read_parquet(os.getcwd()+'/data_buffers/daily_subset_part_{}'.format(i))
    combined_df.drop(['daily_consumption','currreaddate'],1,inplace=True)
    
    # Add primary dataset features
    combined_df = combined_df.merge(df_subset, on=['meterref','prevreaddate'], how='left')
    
    # Apply exclusions
    combined_df = combined_df[(combined_df['reading_lag_prev'] >= 0) & (combined_df['reading_lag_prev'] < 90)] # to bridge gap of small missing reading windows
    combined_df = combined_df[combined_df['reading_lag_prev_start'] <= 200] # Select cases where previous reading window was not too long ago
    combined_df = combined_df[(combined_df['daily_consumption'] >= 0) & (combined_df['daily_consumption'] < 200)] # Ensure no 99999 and other corruped readings
    combined_df = combined_df[(combined_df['prev_daily_consumption'] >= 0) & (combined_df['prev_daily_consumption'] < 200)]# Ensure no 99999 and other corruped readings
    
    # Add weather features
    combined_df = combined_df.merge(weather_internal, how='left',on='dateofread_dt')
    
    # Add static features
    for c in ['b_class','age','type','storeys','classdesc','primarydesc','secondarydesc','tertiarydesc',
              'quaternarydesc','acorntype', 'acorncat', 'acorngroup']:
        combined_df = pd.concat([combined_df, pd.get_dummies(combined_df[c], prefix=c)], 1)
        combined_df.drop(c, 1, inplace=True)
        
    combined_df.columns = pp.pythonify_cols(combined_df.columns)
        
    # Ensure all variable names are in each chunk of data. Use with caution.
    for feat in all_feats:
        if feat not in combined_df:
            combined_df[feat] = 0
    
    # Predict new daily values based on the models built
    combined_df_list = []
    for date, date_ in zip(modelling_window_dates, modelling_window_dates_date_format):
        print('{}   '.format(date), end='\r')
        combined_df_subset = combined_df[combined_df.dateofread_dt == date_].copy()
        model = pp.load_obj(os.getcwd()+'/models/model_{}.pkl'.format(date), full_path=True)[date]['model']
        combined_df_subset['predicted'] = model.predict(combined_df_subset[model.get_booster().feature_names])
        combined_df_list.append(combined_df_subset)
    combined_df = pd.concat(combined_df_list)
    
    # Redistribute the actual meter read values according to the daily predicted values distribution and remove negative xgb residuals
    temp = combined_df[['meterref','prevreaddate','daily_consumption', 'dateofread_dt', 'predicted']].copy()
    temp['above_average'] = temp['predicted'] > temp['daily_consumption']
    temp['negative'] = temp['predicted'] < 0
    temp = temp.merge(temp[temp['negative']].groupby(['meterref','prevreaddate'])['predicted'].min().rename('neg_min').astype(np.float32).reset_index(), how='left')
    temp['predicted'] += temp['neg_min'].fillna(0).abs()
    temp = temp.merge(temp.groupby(['meterref', 'prevreaddate'])['daily_consumption'].sum().rename('total_consumption').reset_index(), how='left')
    temp = temp.merge(temp.groupby(['meterref', 'prevreaddate'])['predicted'].sum().rename('pred_sum').reset_index(), how='left')
    temp['pred_distribution'] = temp['predicted'] / temp['pred_sum']
    temp['new_daily_consumption'] = temp['total_consumption'] * temp['pred_distribution']
    combined_df = combined_df.merge(temp[['meterref','prevreaddate','new_daily_consumption', 'dateofread_dt']], how='left',on=['meterref','prevreaddate', 'dateofread_dt'])
    del temp

    combined_df.to_parquet(os.getcwd()+'/data_buffers/daily_adjusted_subset_part_{}'.format(i))
    
print('Finished featurisation. Time elapsed: {}{}'.format((pd.Timestamp.now() - start) / pd.Timedelta(1, 'm'), ' '*20))

In [ ]:
%reset -f

In [ ]:
# %%writefile generate_final_dataset.py

import pandas as pd
import numpy as np
import phclib as pp
import os
import warnings
warnings.filterwarnings("ignore")

print('Merging files and generating final subsampled dataset...')
start = pd.Timestamp.now()

meters_list = pp.load_obj('meterref_split_list')
sample_days_per_month = 5
pre_covid_frac = 1

# Due to limited data since the beginning of COVID19 we subsample from pre-covid period only in order to fit data into memory
for i, meters in enumerate(meters_list):
    print('{}/{}   '.format(i+1,len(meters_list)), end='\r')
    if i == 0:
        df_daily = pd.read_parquet(os.getcwd()+'/data_buffers/daily_adjusted_subset_part_{}'.format(i)).drop(['currreaddate'], 1)
        
        # Sample 5 random days per meter per month
        df_daily['month'] = df_daily['dateofread_dt'].dt.month.astype('int16')
        df_daily = df_daily.sample(frac=1)
        df_daily = df_daily.groupby(['meterref','month']).head(5)
        df_daily.drop('month',1, inplace=True)
        
        # All the ints are dummy variables so we reduce memory requirements
        for feat in df_daily.select_dtypes(include=['int64', 'int32', 'int16', 'int8']):
            df_daily[feat] = df_daily[feat].astype(np.uint8)
        df_daily = pp.compress_df(df_daily)
        
        # Split data to pre- and post-isolation in order to retain the data at that time
        df_daily_post_mar_2020 = df_daily[df_daily.dateofread_dt >= '2020-03-18']
        covid_data = df_daily_post_mar_2020[df_daily_post_mar_2020.dateofread_dt >= '2020-06-01']['meterref'].unique()
        df_daily_pre_mar_2020 = df_daily[df_daily.dateofread_dt < '2020-03-18']
        df_daily_pre_mar_2020_covid = df_daily_pre_mar_2020[df_daily_pre_mar_2020['meterref'].isin(covid_data)]
        df_daily_pre_mar_2020_ncovid = df_daily_pre_mar_2020[~df_daily_pre_mar_2020['meterref'].isin(covid_data)]
        df_daily = pd.concat([df_daily_post_mar_2020, df_daily_pre_mar_2020_covid, df_daily_pre_mar_2020_ncovid.sample(frac=pre_covid_frac)])
        
        del df_daily_post_mar_2020, df_daily_pre_mar_2020
    else:
        df_daily_temp = pd.read_parquet(os.getcwd()+'/data_buffers/daily_adjusted_subset_part_{}'.format(i)).drop(['currreaddate'], 1)
        
        # Sample 5 random days per meter per month
        df_daily_temp['month'] = df_daily_temp['dateofread_dt'].dt.month.astype('int16')
        df_daily_temp = df_daily_temp.sample(frac=1)
        df_daily_temp = df_daily_temp.groupby(['meterref','month']).head(5)
        df_daily_temp.drop('month',1, inplace=True)
        
        
        # All the ints are dummy variables so we reduce memory requirements
        for feat in df_daily_temp.select_dtypes(include=['int64', 'int32', 'int16', 'int8']):
            df_daily_temp[feat] = df_daily_temp[feat].astype(np.uint8)
            
        # Split data to pre- and post-isolation in order to retain the data at that time
        df_daily_temp = pp.compress_df(df_daily_temp)
        df_daily_temp_post_mar_2020 = df_daily_temp[df_daily_temp.dateofread_dt >= '2020-03-18']
        df_daily_temp_pre_mar_2020 = df_daily_temp[df_daily_temp.dateofread_dt < '2020-03-18']
        covid_data = df_daily_temp_post_mar_2020[df_daily_temp_post_mar_2020.dateofread_dt >= '2020-06-01']['meterref'].unique()
        df_daily_pre_mar_2020_covid = df_daily_temp_pre_mar_2020[df_daily_temp_pre_mar_2020['meterref'].isin(covid_data)]
        df_daily_pre_mar_2020_ncovid = df_daily_temp_pre_mar_2020[~df_daily_temp_pre_mar_2020['meterref'].isin(covid_data)]
        df_daily_temp = pd.concat([df_daily_temp_post_mar_2020, df_daily_pre_mar_2020_covid, df_daily_pre_mar_2020_ncovid.sample(frac=pre_covid_frac)])
        
        df_daily = pd.concat([df_daily, df_daily_temp])
        del df_daily_temp_post_mar_2020, df_daily_temp_pre_mar_2020, df_daily_temp,df_daily_pre_mar_2020_covid,df_daily_pre_mar_2020_ncovid
        
# Add seasonality and covid-specific features
misc_variables = pp.compress_df(pd.read_parquet('misc_data_preprocessed.pqt'))
df_daily = df_daily.merge(misc_variables, how='left',on='dateofread_dt')
df_daily['month'] = df_daily['dateofread_dt'].dt.month
df_daily = pd.concat([df_daily, pd.get_dummies(df_daily['month'], prefix='month')], 1).drop('month',1)

# Add rolling weather variables to identify persisting weather trends
weather_internal = pd.read_parquet('weather_data.pqt').sort_values('dateofread_dt')
for c in weather_internal.drop('dateofread_dt',1):
    weather_internal['{}_rolling7'.format(c)] = weather_internal[c].rolling(7).mean().bfill().reset_index(drop=True).astype(np.float32)
    weather_internal['{}_rolling21'.format(c)] = weather_internal[c].rolling(21).mean().bfill().reset_index(drop=True).astype(np.float32)
    weather_internal.drop(c, 1, inplace=True)
    
df_daily = df_daily.merge(weather_internal, how='left', on='dateofread_dt')
        
print('Saving to disk. Data shape: {} / Time Elapsed: {} minutes{}'.format(df_daily.shape, (pd.Timestamp.now() - start) / pd.Timedelta(1, 'm'), ' '*20))
df_daily.to_parquet('full_daily_pcc_dataset.pqt')

In [ ]:
%reset -f

In [ ]:
# %%writefile general_daily_model.py

from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
import pandas as pd
import numpy as np
import phclib as pp
import pickle
import os
import warnings
warnings.filterwarnings("ignore")

# Build general model without recent consumption figures in order to be able to separate out COVID impact

print('Training general model on the final daily dataset')
start = pd.Timestamp.now()

# Load relevant data
model_data = pd.read_parquet('full_daily_pcc_dataset.pqt')

# Downsample to about 20% to fit into memory

model_data['month'] = model_data['dateofread_dt'].dt.month.astype('int16')
model_data = model_data.sample(frac=1)
model_data = model_data.groupby(['meterref','month']).head(1)
model_data.drop('month',1, inplace=True)
model_data = model_data[model_data.dateofread_dt < '2020-02-01 00:00:00']

model_data['new_daily_consumption'] = model_data['new_daily_consumption'].fillna(0)

print('Modelling data shape: ', model_data.shape)

exclude = ['prevreaddate','meterref','dateofread_dt','currreaddate','daily_consumption',
           'new_daily_consumption', 'reading_window_days', 'reading_lag_prev_start','predicted',
           'prev_daily_consumption', 'cum_prev_daily_consumption', 'cum_prev_daily_consumption2']
use_cols = [c for c in model_data if c not in exclude]

# Create train / test split
X_train, X_test, y_train, y_test = pp.train_test_split_by_key(model_data,
                                                           target='new_daily_consumption',
                                                           usecols=use_cols,
                                                           key='meterref',
                                                           test_size=0.5)

del model_data
import gc
gc.collect()

XGB_model = XGBRegressor(learning_rate=0.05,
                         max_depth=5,
                         n_estimators=500,
                         n_jobs=31,
                         objective='reg:squarederror')

# Fit with automated stopping based on the test set
print('Fit start')
print(X_train.shape)
XGB_model.fit(X_train,
              y_train,
              verbose=True,
              early_stopping_rounds=20,
              eval_metric='rmse',
              eval_set=[(X_test, y_test)])

train_score = XGB_model.score(X_train, y_train)
test_score = XGB_model.score(X_test, y_test)
print('Train score: {} \nTest score: {}'.format(train_score, test_score))

print('Saving to disk. Time Elapsed: {}'.format((pd.Timestamp.now() - start) / pd.Timedelta(1, 'm')))
pp.save_obj(XGB_model, 'raw_daily_xgb_pcc_model')

In [ ]:
%reset -f

In [ ]:
# %%writefile redistribute_noncovid_consumption.py

import pandas as pd
import numpy as np
import phclib as pp
import os
import warnings
warnings.filterwarnings("ignore")

print('Redistributing consumption after covid impact removal...')
start = pd.Timestamp.now()

# Remove covid impact by predicting with nullified covid variables and applying the percentage change to the daily consumption

# Load data and downsample
model_data = pd.read_parquet('full_daily_pcc_dataset.pqt')
model_data['month'] = model_data['dateofread_dt'].dt.month.astype('int16')
model_data = model_data.sample(frac=1)
model_data = model_data.groupby(['meterref','month']).head(1)
model_data.drop('month',1, inplace=True)

XGB_model = pp.load_obj('raw_daily_xgb_pcc_model')
misc_variables = pp.compress_df(pd.read_parquet('misc_data_preprocessed.pqt')).columns

# Predict based on trained model
model_data['predicted'] = XGB_model.predict(model_data[XGB_model.get_booster().feature_names])

# Nullify covid variables
for c in ['new_covid_deaths','new_hospital_admissions','new_covid_cases']:
    model_data[c] = np.NaN

for c in ['is_covid_period'] + [c for c in misc_variables if 'covid_lockdown_stage' in c]:
    model_data[c] = 0
    
model_data['predicted_noncovid'] = XGB_model.predict(model_data[XGB_model.get_booster().feature_names])

# Generate covid impact stats
cvd_meters = model_data[(model_data['dateofread_dt'] >= '2020-06-01 00:00:00')].meterref.unique()
cvd_set = model_data[(model_data['dateofread_dt'] >= '2020-03-01 00:00:00') & model_data['meterref'].isin(cvd_meters)]
cvd_impact = (cvd_set['predicted'].sum() - cvd_set['predicted_noncovid'].sum()) / cvd_set['predicted_noncovid'].sum()
pp.save_obj(cvd_impact, 'covid_impact_percentage')

print("Covid impact: ", cvd_impact)

model_data['cvd_impact'] = model_data['predicted_noncovid'] / model_data['predicted']
model_data.loc[model_data['dateofread_dt'] >= '2020-03-01 00:00:00', 'new_daily_consumption'] = model_data.loc[model_data['dateofread_dt'] >= '2020-03-01 00:00:00', 'new_daily_consumption'] * model_data.loc[model_data['dateofread_dt'] >= '2020-03-01 00:00:00', 'cvd_impact']
model_data.drop(['predicted','predicted_noncovid','cvd_impact'], 1).to_parquet('full_daily_pcc_dataset_noncovid.pqt')

In [ ]:
%reset -f

In [ ]:
# %%writefile final_model.py

from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
import xgboost
import shap
import pandas as pd
import numpy as np
import phclib as pp
import pickle
import os
import warnings
warnings.filterwarnings("ignore")

print('Training final clean model for ongoing phc calculations...')
start = pd.Timestamp.now()

# Build final model without the impact of covid

# Load relevant data
model_data = pd.read_parquet('full_daily_pcc_dataset_noncovid.pqt').sample(frac=0.75)
model_data['new_daily_consumption'] = model_data['new_daily_consumption'].fillna(0)

misc_variables_cols = pd.read_parquet('misc_data_preprocessed.pqt').columns

# Exclude COVID-related features apart from normal exclusions
exclude = ['prevreaddate','meterref','dateofread_dt','currreaddate','daily_consumption', 'new_daily_consumption', 'reading_window_days']
exclude.extend(['new_covid_deaths','new_hospital_admissions','new_covid_cases'])
exclude.extend(['is_covid_period'] + [c for c in misc_variables_cols if 'covid_lockdown_stage' in c])
model_data.columns = pp.pythonify_cols(model_data.columns)
use_cols = [c for c in model_data if c not in exclude]

# Create train / test split
X_train, X_test, y_train, y_test = train_test_split(model_data[use_cols], model_data['new_daily_consumption'],
                                                    test_size=0.5, random_state=1)

# Free up memory
del model_data


# Train model
XGB_model = XGBRegressor(learning_rate=0.05,
                         max_depth=5,
                         n_estimators=750,
                         n_jobs=31,
                         objective='reg:squarederror')

XGB_model.fit(X_train,
              y_train,
              verbose=True,
              early_stopping_rounds=20,
              eval_metric='rmse',
              eval_set=[(X_test, y_test)])

train_score = XGB_model.score(X_train, y_train)
test_score = XGB_model.score(X_test, y_test)

print('Train score: {} \nTest score: {}'.format(train_score, test_score))
print('Saving to disk. Time Elapsed: {}'.format((pd.Timestamp.now() - start) / pd.Timedelta(1, 'm')))

pp.save_obj(XGB_model, 'daily_xgb_phc_model')

print('\nGenerating model validation...\n')
print('1. Permutation Feature Importance (see eli5 documentation).')
print('2. Shapley scores feature importance (Efficiency, Symmetry, Dummy and Additivity) and effect on model outputs.')

#Xgboost feature importance

fig, ax = plt.subplots(figsize=(10, 30))
xgboost.plot_importance(XGB_model, max_num_features=100, ax=ax)
plt.savefig('feat_importances.png')

# Shap scores model interpretation
shap_x_train = X_train[XGB_model.get_booster().feature_names].sample(frac=0.5)

# Create our SHAP explainer
shap_explainer = shap.TreeExplainer(XGB_model)

# Calculate the shapley values for our data
shap_values = shap_explainer.shap_values(shap_x_train)
shap.summary_plot(shap_values, shap_x_train,auto_size_plot=True)
plt.savefig('shap_summary.png')